Initializing staff

In [17]:
import numpy as np
import scipy.integrate as integrate
import lhapdf

Setting up constants

In [2]:
s_h = 27**2 # in GeV (TeV)
p_t = [1,2,3,4]#[0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00] #GeV
i_ = [-4,-3,-2,-1,1,2,3,4]
j_ = [-4,-3,-1,-2,2,1,3,4]
alpha_em=1./137

Setting up pdf sets

In [3]:
pdfs = [lhapdf.mkPDF("NNPDF23_nlo_as_0118", 0), lhapdf.mkPDF("nNNPDF20_nlo_as_0118_D2", 0), lhapdf.mkPDF("cteq6l1", 0)]

LHAPDF 6.5.4 loading /home/yoren/bnl/lhapdf/LHAPDF-6.5.4/install/share/LHAPDF/NNPDF23_nlo_as_0118/NNPDF23_nlo_as_0118_0000.dat
NNPDF23_nlo_as_0118 PDF set, member #0, version 6; LHAPDF ID = 229800
LHAPDF 6.5.4 loading /home/yoren/bnl/lhapdf/LHAPDF-6.5.4/install/share/LHAPDF/nNNPDF20_nlo_as_0118_D2/nNNPDF20_nlo_as_0118_D2_0000.dat
nNNPDF20_nlo_as_0118_D2 PDF set, member #0, version 1; LHAPDF ID = 30010300
LHAPDF 6.5.4 loading /home/yoren/bnl/lhapdf/LHAPDF-6.5.4/install/share/LHAPDF/cteq6l1/cteq6l1_0000.dat
cteq6l1 PDF set, member #0, version 4; LHAPDF ID = 10042


Function for Compton and annihilation cross-section

In [4]:
def compton_anih_crossection(y_1, y_2, p, iset1, iset2, isospin1, isospin2):

    
    if abs(y_1)>2 or y_2>2: return 0.
        
    x_1 = (2*p/np.sqrt(s_h))* np.exp((y_1 + y_2)/2)*np.cosh(abs((y_1-y_2)/2))
    x_2 = (2*p/np.sqrt(s_h))* np.exp(-(y_1 + y_2/2))*np.cosh(abs((y_1-y_2)/2))
    s = x_1*x_2*s_h
    
    if np.all( [x_1 > 0 , x_1 < 1 , x_2 > 0 , x_2 < 1, 1. - 4*(p)**2/s > 0] ):
        
        cos_theta = np.sqrt(1 - 4*(p)**2/s)
        u = (-1*s/2)*(1 + cos_theta)
        t = (-1*s/2)*(1 - cos_theta)
        Q2 = -t
        
        pdf1 = pdfs[iset1]
        pdf2 = pdfs[iset2]
        alpha_s = 0.5*(pdf1.alphasQ2(Q2)+pdf2.alphasQ2(Q2))

        sigma=0.
        for iquark in range(len(i_)):
            i1, i2 = i_[iquark], i_[iquark]
            if isospin1==1: i1 = j_[iquark]
            if isospin2==1: i2 = j_[iquark]
            
            eq2 = (1. + (i_[iquark]+1)%2)**2/9.
            
            pdf_i = max(pdf1.xfxQ2(i1, x_1, Q2)/x_1,0.)
            pdf_j = max(pdf2.xfxQ2(21, x_2, Q2)/x_2,0.)
            pdf_k = max(pdf2.xfxQ2(-i2, x_2, Q2)/x_2,0.)
            pdf_i2 = max(pdf2.xfxQ2(i2, x_2, Q2)/x_2,0.)
            pdf_j2 = max(pdf1.xfxQ2(21, x_1, Q2)/x_1,0.)
        
            sigma_compton_dt = -eq2*(8*np.pi*alpha_s*alpha_em/s)*((s**2 + u**2)/(4*u*s))

            sigma_anihhil_dt = -eq2*(8*np.pi*alpha_s*alpha_em/s)*((t**2 + u**2)/(4*t*s))
            
            sigma += pdf_i*pdf_j*sigma_compton_dt + pdf_i2*pdf_j2*sigma_compton_dt
            sigma += pdf_i*pdf_k*sigma_anihhil_dt
        
        return  sigma
    return 0.

Testing

In [5]:
print(compton_anih_crossection(0.1,0.6,1,0,0,0,0))

2.4164622967036413


Integration Bounds and accuracy

In [6]:
def bounds_y(p, iset1, iset2, isospin1, isospin2):
    return [np.log(p/np.sqrt(s_h)), np.log(np.sqrt(s_h)/p)]
        
def bounds_x(y_1, p, iset1, iset2, isospin1, isospin2):
    return [np.log(p/(np.sqrt(s_h) - p*np.exp(-y_1))), np.log((np.sqrt(s_h) - p*np.exp(-y_1))/p)]
    
def opts0(y_1, p, iset1, iset2, isospin1, isospin2):
    return {'epsrel' : 0.05}

def opts1(p, iset1, iset2, isospin1, isospin2):
    return {'epsrel' : 0.05}                

Integration function

In [7]:
def integration_cs(pT,iset1,iset2,isospin1,isopin2):
    result=[]
    for p in pT:
        integral = integrate.nquad(compton_anih_crossection, [bounds_x, bounds_y], args=[p,iset1,iset2,isospin1,isopin2], opts=[opts0,opts1])
        #print(p,i,integral)
        result.append(integral)   
                
    return result

Parallel calculating for $p+p$

In [8]:
import multiprocess as mp

Ntr = 4
pool = mp.Pool(Ntr)

hist_array = pool.starmap(integration_cs, ([[1.0*i+1.0], 0,0,0,0] for i in range(Ntr)))
    
pool.close()   

Parallel calculating for $p+n$

In [9]:
pool_pn = mp.Pool(Ntr)

hist_array_pn = pool_pn.starmap(integration_cs, [([1.0*i+1.0], 0,0,0,1) for i in range(Ntr)])
    
pool.close()   

Parallel calculating for $n+p$

In [10]:
pool_np = mp.Pool(Ntr)

hist_array_np = pool_np.starmap(integration_cs, [([1.0*i+1.0], 0,0,1,0) for i in range(Ntr)])
    
pool.close()   

Parallel calculating for $n+n$

In [11]:
pool_nn = mp.Pool(Ntr)

hist_array_nn = pool_nn.starmap(integration_cs, [([1.0*i+1.0], 0,0,1,1) for i in range(Ntr)])
    
pool.close()   

Parallel calculating for $d+d$

In [12]:
pool_dd = mp.Pool(Ntr)

hist_array_dd = pool_dd.starmap(integration_cs, [([1.0*i+1.0], 1,1,0,0) for i in range(Ntr)])
    
pool.close()   

In [13]:
pool_pd = mp.Pool(Ntr)

hist_array_pd = pool_pd.starmap(integration_cs, [([1.0*i+1.0], 1,0,0,0) for i in range(Ntr)])
    
pool.close()   

pool_dp = mp.Pool(Ntr)

hist_array_dp = pool_dp.starmap(integration_cs, [([1.0*i+1.0], 0,1,0,0) for i in range(Ntr)])
    
pool.close()   

In [14]:
print(hist_array_pd)
print(hist_array_dp)

[[(6.120411956338261, 0.2824276894058063)], [(0.12031521178709798, 0.005720324087804984)], [(0.005409203851333432, 0.0002087106486660701)], [(0.00039276945499486254, 1.924323675426784e-05)]]
[[(6.179615537969275, 0.19723350849016086)], [(0.12010946517983526, 0.004210482455275937)], [(0.005436165939105557, 0.0002636048369112794)], [(0.0003923287803443472, 1.8966918106233875e-05)]]


Factors

In [15]:
print(hist_array,hist_array_dd)
res_iso=[]
for ival in range(len(hist_array)):
    res_iso.append(0.25*(hist_array[ival][0][0]+hist_array_pn[ival][0][0]+hist_array_np[ival][0][0]+hist_array_nn[ival][0][0])/(hist_array[ival][0][0]+1e-20))
print("pT:1.0,2.0,3.0,4.0",res_iso)
res=[]
for ival in range(len(hist_array)):
    res.append(hist_array_dd[ival][0][0]/(hist_array[ival][0][0]+1e-20))
print("pT:1.0,2.0,3.0,4.0",res)

[[(5.857437480139133, 0.18418552392841825)], [(0.1342169756631816, 0.0047892679951132625)], [(0.00621643860988663, 0.0002781600690119949)], [(0.0004538280277397278, 2.122796718355055e-05)]] [[(6.482395457635013, 0.16208554004409947)], [(0.10703809900881613, 0.005296057733247661)], [(0.004731190356509968, 0.00018889965930380856)], [(0.00033621593497026246, 1.665287410350277e-05)]]
pT:1.0,2.0,3.0,4.0 [0.9198211427449647, 0.8570427046865007, 0.819755980955528, 0.7886899600489214]
pT:1.0,2.0,3.0,4.0 [1.1066947755934111, 0.7975004538727571, 0.7610773070268044, 0.7408443604613235]
